# Análise de Variáveis para Previsão do PIB com Modelos Autoregressivos

**Objetivo:**

Nosso objetivo é prever o Produto Interno Bruto (PIB) utilizando modelos de regressão linear. Em particular, exploramos um modelo autoregressivo, onde utilizamos os próprios valores passados do PIB (defasagens, ou lags) como variáveis explicativas.

**Metodologia:**

Carregamos um conjunto de dados contendo informações sobre o PIB e outras variáveis econômicas. Para avaliar a relação do PIB com seus valores passados, criamos defasagens temporais (lags), ou seja, colunas que representam o PIB de meses anteriores (PIBi1, PIBi2, ..., PIBi11).

**Avaliação:**

Para avaliar a qualidade da previsão, utilizamos o Erro Quadrático Médio (MSE - Mean Squared Error), que mede a diferença entre os valores previstos e os valores reais do PIB. Quanto menor o MSE, melhor o modelo.

In [15]:
import pandas as pd
from sklearn.linear_model import LinearRegression
# biblioteca com métricas de erro de previsão. Escolhemos a mse
from sklearn.metrics import mean_squared_error
import numpy as np
import itertools

In [2]:
# Carga de dados
df = pd.read_csv("https://raw.githubusercontent.com/diogenesjusto/FIAP/master/SHIFT/Data/pib.csv")

In [7]:
df.columns

Index(['Unnamed: 0', 'ANO_MES', 'PIB', 'BRL', 'BRP', 'BRT', 'SLP', 'SPP',
       'SPT', 'PRL', 'PRP', 'PRT', 'RJL', 'RJP', 'RJT', 'D1', 'D2', 'D3', 'D4',
       'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'PIBi1', 'PIBi2', 'PIBi3',
       'PIBi4', 'PIBi5', 'PIBi6', 'PIBi7', 'PIBi8', 'PIBi9', 'PIBi10',
       'PIBi11', 'PIBi12'],
      dtype='object')

In [3]:
# Separação de treino e teste
treino = df.iloc[0:126]
teste = df.iloc[126:138]

In [13]:
resultados = {}

# Regressão
Y_treino = treino.iloc[:, [2]]  # PIB

# Loop para testar cada variável de índice 26 a 36
for i in range(26, 37):
    X = [df.columns[i]]  # Seleciona a variável pela posição
    X_treino = treino.loc[:, X]
    X_teste = teste.loc[:, X]

    # Ajuste do modelo
    modelo = LinearRegression()
    modelo.fit(X_treino, Y_treino)

    # Previsões
    p = modelo.predict(X_teste)

    # Métrica de erro
    mse = mean_squared_error(teste.PIB, p)

    # R² do modelo
    r2 = modelo.score(X_teste, teste.PIB)

    # R (correlação)
    R = np.sqrt(r2) if r2 >= 0 else -np.sqrt(-r2)  # Garante que R fique coerente

    # Armazena os resultados no dicionário
    resultados[X[0]] = (mse, r2, R)

# Exibir os resultados organizados
print("\nResultados de todas as variáveis testadas:")
print(f"{'Variável':<10} {'MSE':<10} {'R²':<10} {'R':<10}")
print("=" * 40)

for variavel, (mse, r2, R) in resultados.items():
    print(f"{variavel:<10} {mse:<10.4f} {r2:<10.4f} {R:<10.4f}")

# Identificar a melhor variável (menor MSE)
melhor_variavel = min(resultados, key=lambda k: resultados[k][0])
melhor_mse, melhor_r2, melhor_R = resultados[melhor_variavel]

print("\nMelhor variável encontrada:")
print(f"Variável: {melhor_variavel}")
print(f"MSE: {melhor_mse:.4f}")
print(f"R²: {melhor_r2:.4f}")
print(f"R: {melhor_R:.4f}")


Resultados de todas as variáveis testadas:
Variável   MSE        R²         R         
PIBi1      13.9076    -0.1451    -0.3809   
PIBi2      11.2472    0.0739     0.2719    
PIBi3      16.4709    -0.3562    -0.5968   
PIBi4      22.7694    -0.8748    -0.9353   
PIBi5      18.8877    -0.5552    -0.7451   
PIBi6      25.5165    -1.1010    -1.0493   
PIBi7      25.7846    -1.1230    -1.0597   
PIBi8      25.9337    -1.1353    -1.0655   
PIBi9      28.8140    -1.3725    -1.1715   
PIBi10     24.3253    -1.0029    -1.0014   
PIBi11     28.4415    -1.3418    -1.1584   

Melhor variável encontrada:
Variável: PIBi2
MSE: 11.2472
R²: 0.0739
R: 0.2719


**Como interpretar o MSE?**

✅ Quanto menor o MSE, melhor o modelo, pois significa que os valores previstos estão mais próximos dos valores reais.

❌ Valores altos de MSE indicam previsões ruins, pois os erros ao prever os valores reais são grandes.


A variável **PIBi2** teve o **menor MSE (11.2472)**, então ela fez a **melhor previsão do PIB** comparada às outras defasagens testadas. Isso sugere que o PIB com 2 meses de lag tem a **melhor relação para prever o PIB atual**, pelo menos **dentro desse modelo simples**.

**Conclusão**:

A autoregressão mostrou-se uma abordagem válida para prever o PIB, e os resultados sugerem que o PIB de dois meses atrás (lag 2) é o melhor preditor individual do PIB atual.

**Próximos Passos:**

Como próximos passos, podemos testar modelos que consideram múltiplos valores passados do PIB simultaneamente.

Esse estudo reforça a importância da análise temporal na previsão econômica e destaca como a dependência entre períodos pode ser explorada para melhorar a modelagem estatística.

## PARES DE VARIÁVEIS PIB LAGs

In [18]:
resultados2 = {}

# Regressão
Y_treino = treino.iloc[:, [2]]  # PIB

# Gerar todos os pares possíveis de variáveis de índice 26 a 36
variaveis = df.columns[26:37]  # Seleciona as variáveis de índice 26 a 36

# Gerar todos os pares possíveis
pares_variaveis = itertools.combinations(variaveis, 2)

# Loop para testar cada par de variáveis
for par in pares_variaveis:
    X = list(par)  # Converte o par em uma lista de variáveis
    X_treino = treino.loc[:, X]
    X_teste = teste.loc[:, X]

    # Ajuste do modelo
    modelo = LinearRegression()
    modelo.fit(X_treino, Y_treino)

    # Previsões
    p = modelo.predict(X_teste)

    # Métrica de erro
    mse = mean_squared_error(teste.PIB, p)

    # R² do modelo
    r2 = modelo.score(X_teste, teste.PIB)

    # R (correlação)
    R = np.sqrt(r2) if r2 >= 0 else -np.sqrt(-r2)  # Garante que R fique coerente

    # Armazena os resultados2 no dicionário
    resultados2[str(par)] = (mse, r2, R)

# Exibir os resultados2 organizados
print("\nresultados2 de todos os pares de variáveis testados:")
print(f"{'Variável(s)':<20} {'MSE':<10} {'R²':<10} {'R':<10}")
print("=" * 60)

for variaveis, (mse, r2, R) in resultados2.items():
    print(f"{variaveis:<20} {mse:<10.4f} {r2:<10.4f} {R:<10.4f}")

# Identificar o melhor par de variáveis (menor MSE)
melhor_par = min(resultados2, key=lambda k: resultados2[k][0])
melhor_mse, melhor_r2, melhor_R = resultados2[melhor_par]

print("\nMelhor par de variáveis encontrado:")
print(f"Variável(s): {melhor_par}")
print(f"MSE: {melhor_mse:.4f}")
print(f"R²: {melhor_r2:.4f}")
print(f"R: {melhor_R:.4f}")


resultados2 de todos os pares de variáveis testados:
Variável(s)          MSE        R²         R         
('PIBi1', 'PIBi2')   9.1339     0.2479     0.4979    
('PIBi1', 'PIBi3')   12.7611    -0.0507    -0.2252   
('PIBi1', 'PIBi4')   13.5610    -0.1166    -0.3414   
('PIBi1', 'PIBi5')   11.7979    0.0286     0.1691    
('PIBi1', 'PIBi6')   13.5399    -0.1148    -0.3389   
('PIBi1', 'PIBi7')   12.7474    -0.0496    -0.2227   
('PIBi1', 'PIBi8')   12.5054    -0.0297    -0.1722   
('PIBi1', 'PIBi9')   13.5267    -0.1138    -0.3373   
('PIBi1', 'PIBi10')  11.6754    0.0387     0.1967    
('PIBi1', 'PIBi11')  12.8673    -0.0595    -0.2438   
('PIBi2', 'PIBi3')   10.0048    0.1762     0.4198    
('PIBi2', 'PIBi4')   11.1404    0.0827     0.2876    
('PIBi2', 'PIBi5')   10.9078    0.1019     0.3192    
('PIBi2', 'PIBi6')   11.1136    0.0849     0.2914    
('PIBi2', 'PIBi7')   11.7879    0.0294     0.1715    
('PIBi2', 'PIBi8')   11.8044    0.0281     0.1675    
('PIBi2', 'PIBi9')   13.6012

**Explicação dos Resultados Obtidos**:

Após testar diversos pares de variáveis "Lag" (defasagens do PIB), encontramos que o melhor par de variáveis para explicar o PIB atual foi ('PIBi1', 'PIBi2')

Acreditamos que a combinação de lag PIBi1 e PIBi2 ajuda a capturar uma visão mais completa do comportamento do PIB.

*  O PIBi1 reflete reações imediatas
*  O PIBi2 captura um comportamento mais suavizado


Portanto, dando ao modelo uma visão mais equilibrada das forças econômicas que impactam o PIB